In [26]:
import pandas as pd

result_df = pd.read_csv("/home/result.csv")

In [27]:
result_df

,Job title,website,company name,location,area,posted time,classification,salary,classification1,classification2,classification3,low_salary,high_salary,Domain
0,Administrator,https://www.seek.co.nz/job/50582301?type=promo...,Private Advertiser,Bay of Plenty,Tauranga,0,Office Management,unknown,Administration,Office SupportAdministration,Office Support,53476.0,53476.0,Admin
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promo...,Avenues Orthodontics,Bay of Plenty,Tauranga,0,Receptionists,unknown,Administration,Office SupportAdministration,Office Support,40125.0,44621.0,Admin
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=stand...,New Zealand Police,Auckland,unknown,-4,Other,unknown,Administration,Office SupportAdministration,Office Support,36073.0,37362.0,Admin
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=stand...,Kew Pacific Island Early Learning Centre,Southland,Invercargill,0,Administrative Assistants,unknown,Administration,Office SupportAdministration,Office Support,44309.0,47447.0,Admin
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=stand...,Private Advertiser,Canterbury,Christchurch,-4,Client & Sales Administration,unknown,Administration,Office SupportAdministration,Office Support,37304.0,37304.0,Admin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16886,Research Fellow in Physiology - Faculty of Med...,https://www.seek.co.nz/job/50584769?type=stand...,The University of Auckland,Auckland,Auckland Central,-12,Research & Fellowships,unknown,Education,TrainingEducation,Training,53113.0,53113.0,Education
16887,Early Childhood Teacher - Central Auckland,https://www.seek.co.nz/job/50493625?type=stand...,Randstad - Education,Auckland,unknown,-27,unknown,$25-$29p.h.,unknown,unknown,unknown,40000.0,46400.0,Education
16888,Postdoctoral Fellow,https://www.seek.co.nz/job/50575193?type=stand...,Massey University,Manawatu,Palmerston North,-13,Research & Fellowships,unknown,Education,TrainingEducation,Training,67620.0,67620.0,Education
16889,Early Childhood Teacher - Albany,https://www.seek.co.nz/job/50493623?type=stand...,Randstad - Education,Auckland,"Rodney & North ShoreRodney & North Shore,$25 -...",-27,unknown,$25-$30p.h.,unknown,unknown,unknown,40000.0,48000.0,Education


In [28]:
x_df = result_df[['company name', 'location','posted time', 'low_salary','high_salary']]
y_df = result_df['Domain']

In [29]:
from sklearn.preprocessing import LabelEncoder

x_df = x_df.apply(LabelEncoder().fit_transform)
y_df_encoded = LabelEncoder().fit_transform(y_df.values)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=15)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

xgb_model = xgb.XGBRegressor()

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.01, 0.3), 
    "max_depth": randint(2, 6),
    "n_estimators": randint(100, 600), 
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=12, n_iter=200, cv=3, verbose=1, n_jobs=10, return_train_score=True)

search.fit(x_df, y_df_encoded)

search.cv_results_

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  5.2min
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed: 10.9min
[Parallel(n_jobs=10)]: Done 600 out of 600 | elapsed: 15.2min finished


[06:44:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'mean_fit_time': array([13.86153698, 17.77316149, 13.68321347, 26.77267472, 15.98884002,
        12.48794643, 11.17273998, 16.37037182,  7.56241544,  4.57731128,
         9.26778086, 24.98227612, 15.4880995 ,  7.15652156, 25.74462318,
        23.43363452, 10.46406031,  7.21097612, 11.4681925 ,  7.67851631,
        14.35662715, 23.3991998 , 18.32751973, 18.84511558,  9.38080438,
        12.58539891, 22.69480666,  3.05284333, 15.29875906,  9.31794755,
        19.5750951 , 10.82511306,  3.91328526, 11.89838465, 11.555523  ,
        23.68532713, 22.18181181, 20.97187455, 13.05296373, 32.28970464,
        22.43345777, 22.94641304, 21.1462117 , 12.20810668, 13.17908367,
        23.966736  , 21.70086749, 14.96461391, 19.23040446, 24.40946913,
         9.49176637, 16.28002365,  4.73715361, 14.61550101, 13.33740362,
        12.71581682,  5.6054306 ,  9.40134541, 32.55846898,  7.12414018,
         5.18766459, 14.08023334, 14.95321012, 10.03480776, 21.20620108,
        13.99366792,  7.87396336, 

In [ ]:
import numpy as np
results = search.cv_results_
for i in range(1, 5):
  candidates = np.flatnonzero(results['rank_test_score'] == i)
  for candidate in candidates:
      print("Model with rank: {0}".format(i))
      print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
            results['mean_test_score'][candidate],
            results['std_test_score'][candidate]))
      print("Parameters: {0}".format(results['params'][candidate]))

Model with rank: 1
Mean validation score: -4.960 (std: 3.227)
Parameters: {'colsample_bytree': 0.884096036312531, 'gamma': 0.30919635636966997, 'learning_rate': 0.014006026657373806, 'max_depth': 2, 'n_estimators': 138, 'subsample': 0.9110717808318487}
Model with rank: 2
Mean validation score: -5.186 (std: 4.471)
Parameters: {'colsample_bytree': 0.9095663568333286, 'gamma': 0.1375400345673099, 'learning_rate': 0.016317249855934794, 'max_depth': 3, 'n_estimators': 153, 'subsample': 0.854469382406921}
Model with rank: 3
Mean validation score: -5.246 (std: 5.983)
Parameters: {'colsample_bytree': 0.9862879556873168, 'gamma': 0.2827578577757353, 'learning_rate': 0.2331882112540397, 'max_depth': 2, 'n_estimators': 175, 'subsample': 0.6743581637726378}
Model with rank: 4
Mean validation score: -5.274 (std: 5.893)
Parameters: {'colsample_bytree': 0.9078974006096598, 'gamma': 0.12161556022550185, 'learning_rate': 0.27023088676712, 'max_depth': 3, 'n_estimators': 385, 'subsample': 0.664024992186

In [31]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

def print_evaluate(classifier, y_test, prediction):  
    scores_accuracy = cross_val_score(classifier, x_df, y_df, cv=10, scoring='accuracy')
    scores_f1 = cross_val_score(classifier, x_df, y_df, cv=10, scoring='f1_macro')
    print('K-fold cross-validation results:')
    print(classifier.__class__.__name__+" average accuracy is %2.3f" % scores_accuracy.mean())
    print(classifier.__class__.__name__+" average f1 is %2.3f" % scores_f1.mean())
    print('Classification report: ')
    print(classification_report(y_test, prediction))

In [34]:
params = {'n_estimators': 500,
      'learning_rate': 0.01,
      'max_depth': 5,
      'objective': 'multi:softprob',
      'eval_metric': "auc"}

xgb_model = xgb.XGBClassifier(**params)
xgb_model.fit(X_train, y_train)
#xgb_model.fit(X_train, y_train, early_stopping_rounds=8, eval_set=[(X_test, y_test)])

pred = xgb_model.predict(X_test)
print_evaluate(xgb_model, y_test, pred)

K-fold cross-validation results:
XGBClassifier average accuracy is 0.919
XGBClassifier average f1 is 0.919
Classification report: 
              precision    recall  f1-score   support

       Admin       0.87      0.96      0.92       536
     Banking       0.90      0.97      0.94       772
   Education       0.94      0.94      0.94       874
 Manufacture       0.95      0.86      0.90      1197

    accuracy                           0.92      3379
   macro avg       0.92      0.93      0.92      3379
weighted avg       0.92      0.92      0.92      3379



In [33]:
params = {'n_estimators': 138,
      'learning_rate': 0.014,
      'objective': 'multi:softprob',
      'eval_metric': "auc",
      'colsample_bytree': 0.9,
      'gamma': 0.31,
      'subsample': 0.9,
      'max_depth': 2}

xgb_model = xgb.XGBClassifier(**params)
xgb_model.fit(X_train, y_train)

pred = xgb_model.predict(X_test)
print_evaluate(xgb_model, y_test, pred)

K-fold cross-validation results:
XGBClassifier average accuracy is 0.765
XGBClassifier average f1 is 0.767
Classification report: 
              precision    recall  f1-score   support

       Admin       0.74      0.91      0.82       536
     Banking       0.70      0.91      0.79       772
   Education       0.76      0.66      0.71       874
 Manufacture       0.88      0.70      0.78      1197

    accuracy                           0.77      3379
   macro avg       0.77      0.79      0.77      3379
weighted avg       0.79      0.77      0.77      3379

